In [44]:
import pandas as pd
import requests
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

# Load DataFrame
df = pd.read_csv("extracted_ebooks_info.csv")

In [21]:
import pandas as pd
import requests
from transformers import GPT2Tokenizer
from datasets import Dataset

# Load the DataFrame (replace with actual file path or DataFrame loading code)
df = pd.read_csv('extracted_ebooks_info.csv')

# Function to fetch the text from the link
def fetch_text_from_link(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return ""

# Apply the function to the 'Plain Text Link' column to extract the actual text
df['text'] = df['Plain Text Link'].apply(fetch_text_from_link)

# Optionally, you can add metadata to the text (e.g., Author, Title) if needed
df['formatted_text'] = df.apply(lambda row: f"Author: {row['Author']}\nTitle: {row['Title']}\nRelease Date: {row['Release Date']}\nCategory: {row['Category']}\n{row['text']}", axis=1)

# Now, the DataFrame contains a 'formatted_text' column with the actual text data ready for tokenization


In [41]:
# Create a Dataset
dataset = Dataset.from_pandas(df[['formatted_text']])

# Load the GPT2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Step #5: Add special tokens to the tokenizer (if not already present)
special_tokens_dict = {'additional_special_tokens': ['[AUTHOR]', '[TITLE]', '[RELEASE_DATE]', '[CATEGORY]']}
tokenizer.add_tokens(special_tokens_dict['additional_special_tokens'])

# Resize model's token embeddings to account for the new special tokens
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

Embedding(50261, 768)

In [54]:
# Tokenize the 'formatted_text' column
tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(examples):
    # Tokenize and create the labels (input_ids == labels)
    encodings = tokenizer(examples['formatted_text'], truncation=True, padding="max_length", max_length=512)
    encodings['labels'] = encodings['input_ids']  # Set labels equal to input_ids for causal language modeling
    return encodings

# Apply tokenization
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 437/437 [02:57<00:00,  2.47 examples/s]


In [55]:
# Split the tokenized dataset into training and validation sets using Hugging Face's `train_test_split`
split_dataset = tokenized_datasets.train_test_split(test_size=0.1)

# Convert to DatasetDict for compatibility with Trainer
dataset_dict = DatasetDict({
    'train': split_dataset['train'],
    'validation': split_dataset['test']
})

In [ ]:
# Set up the training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",  # where to save model
    num_train_epochs=3,  # number of training epochs
    per_device_train_batch_size=2,  # batch size for training
    per_device_eval_batch_size=2,  # batch size for evaluation
    warmup_steps=500,  # number of warmup steps for learning rate scheduler
    weight_decay=0.01,  # strength of weight decay
    logging_dir="./logs",  # directory for storing logs
    logging_steps=10,  # log every 10 steps
    evaluation_strategy="steps",  # evaluate every X steps
    save_steps=500,  # save the model every 500 steps
    save_total_limit=2,  # limit the total number of saved checkpoints
)

In [57]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_dict['train'],
    eval_dataset=dataset_dict['validation'],
    tokenizer=tokenizer
)

C:\Users\ethan\AppData\Local\Temp\ipykernel_3564\1201030834.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [58]:
# Fine-tune the model
trainer.train()

# Save the fine-tuned model
trainer.save_model("./fine_tuned_gpt2")


  0%|          | 0/591 [05:39<?, ?it/s]         

{'loss': 4.9699, 'grad_norm': 30.966703414916992, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.05}

























                                                

  0%|          | 0/591 [06:08<?, ?it/s]        



{'eval_loss': 4.2132954597473145, 'eval_runtime': 29.3998, 'eval_samples_per_second': 1.497, 'eval_steps_per_second': 0.748, 'epoch': 0.05}



  0%|          | 0/591 [07:04<?, ?it/s]         

...

{'loss': 0.908, 'grad_norm': 6.631828308105469, 'learning_rate': 5.494505494505495e-07, 'epoch': 2.99}

























                                                   

  0%|          | 0/591 [1:30:18<?, ?it/s]      



{'eval_loss': 1.0958267450332642, 'eval_runtime': 28.6908, 'eval_samples_per_second': 1.534, 'eval_steps_per_second': 0.767, 'epoch': 2.99}



100%|██████████| 591/591 [1:25:39<00:00,  8.70s/it]


{'train_runtime': 5139.0251, 'train_samples_per_second': 0.229, 'train_steps_per_second': 0.115, 'train_loss': 1.73107911376582, 'epoch': 3.0}


In [67]:
# Encode the input prompt
input_text = "Once there was a tree"
inputs = tokenizer.encode(input_text, return_tensors="pt")

# Generate a response
outputs = model.generate(inputs, max_length=150, num_return_sequences=1, repetition_penalty=2.0, temperature=0.7)

# Decode the generated response
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print the result
print(response)

C:\Users\ethan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\generation\configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Once there was a tree, and he had seen it before.
And then came the man who knew nothing of this thing; for that is what we have said in our book: "The Man Who Wasn't There." And so on to another day when they were gone together--and still no one could hear them out --but all these things happened at once with their own hands! They are not known by name or place but as if I'd been told from my mother's house where she kept her children alive (I don' t know how much). But now you see me coming home again? Why did anyone come back after dinner tonight?" Then asked Mr Browning-Smith : "'Why do people stay up here long enough?'
